In [1]:
import torch
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [2]:
data = pd.read_csv("symbol_features_noise.csv")

label_enc = LabelEncoder()
data['modulation_label'] = label_enc.fit_transform(data['modulation_type'])
features = data[['magnitude', 'phase', 'real', 'imag']].values
labels = data['modulation_label'].values

In [3]:
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(features_scaled, labels, test_size=0.2, random_state=42)

X_train, X_test

(array([[ 0.15535884,  0.832327  , -0.31632448,  1.49304673],
        [-1.85607457, -1.29231176, -0.32358172, -0.56616877],
        [ 0.39753286, -0.1482755 ,  1.42358293,  0.02728151],
        ...,
        [ 0.15535884,  1.32262825, -1.23278156,  0.6796084 ],
        [-0.83504684,  1.28176982, -0.87210457,  0.57757496],
        [ 1.40011316,  0.28073809,  1.26111244,  1.44842205]]),
 array([[-0.1714682 , -0.31988094,  1.15535927, -0.39623083],
        [-0.1714682 ,  0.36654081,  0.70803914,  1.13605121],
        [-0.04701193,  1.56777888, -1.29259254,  0.02728151],
        [-1.5264767 ,  1.28176982, -0.65325902,  0.43694644],
        [-0.04701193,  1.56777888, -1.29259254,  0.02728151],
        [-1.06992493, -1.34951357, -0.54651145, -0.78581627],
        [ 0.39753286,  1.28176982, -1.26223032,  0.82826685],
        [-0.83504684, -1.00630269, -0.01574005, -1.07330538],
        [-0.27547906,  0.02332994,  1.1192063 ,  0.43771962],
        [-0.83504684, -1.57832082, -0.87210457, -0.5230

In [5]:
Xtr_tensor = torch.tensor(X_train, dtype=torch.float32)
Xts_tensor = torch.tensor(X_test, dtype=torch.float32)
ytr_tensor = torch.tensor(y_train, dtype=torch.float32)
yts_tensor = torch.tensor(y_test, dtype=torch.float32)

train_dataset = TensorDataset(Xtr_tensor, ytr_tensor)
test_dataset = TensorDataset(Xts_tensor, yts_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [6]:
import torch.nn as nn 
import torch.optim as optim

class ModulationClassifier(nn.Module): 
    def __init__(self, input_size, num_classes): 
        super(ModulationClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(0.3)
    
    def forward(self, x): 
        x = self.relu(self.fc1(x))
        x = self.drop(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

input_size: int = 4
num_classes = len(label_enc.classes_)
model = ModulationClassifier(input_size, num_classes).to(device)

device

'cpu'

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [9]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Ensure labels are of type Long
            labels = labels.long()
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Evaluation function
def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Train and evaluate the model
train_model(model, train_dataloader, criterion, optimizer, num_epochs=100)
evaluate_model(model, test_dataloader)

Epoch [1/100], Loss: 2.0531
Epoch [2/100], Loss: 1.9724
Epoch [3/100], Loss: 1.8824
Epoch [4/100], Loss: 1.7899
Epoch [5/100], Loss: 1.7207
Epoch [6/100], Loss: 1.6542
Epoch [7/100], Loss: 1.5828
Epoch [8/100], Loss: 1.5440
Epoch [9/100], Loss: 1.4913
Epoch [10/100], Loss: 1.4534
Epoch [11/100], Loss: 1.4179
Epoch [12/100], Loss: 1.3884
Epoch [13/100], Loss: 1.3649
Epoch [14/100], Loss: 1.3267
Epoch [15/100], Loss: 1.3069
Epoch [16/100], Loss: 1.2871
Epoch [17/100], Loss: 1.2482
Epoch [18/100], Loss: 1.2214
Epoch [19/100], Loss: 1.2084
Epoch [20/100], Loss: 1.1898
Epoch [21/100], Loss: 1.1693
Epoch [22/100], Loss: 1.1507
Epoch [23/100], Loss: 1.1188
Epoch [24/100], Loss: 1.1053
Epoch [25/100], Loss: 1.0859
Epoch [26/100], Loss: 1.0698
Epoch [27/100], Loss: 1.0500
Epoch [28/100], Loss: 1.0333
Epoch [29/100], Loss: 1.0184
Epoch [30/100], Loss: 0.9949
Epoch [31/100], Loss: 0.9872
Epoch [32/100], Loss: 0.9639
Epoch [33/100], Loss: 0.9657
Epoch [34/100], Loss: 0.9546
Epoch [35/100], Loss: 0